<a href="https://colab.research.google.com/github/seohyun34/test/blob/main/EDA_%EB%B0%8F_%EA%B0%9C%EC%9A%94_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://zindi.africa/competitions/kenya-clinical-reasoning-challenge/data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import os

# EDA

## Train data

In [ ]:
train_raw = pd.read_csv('/content/drive/MyDrive/케냐 임상시험 데이터셋/train_raw.csv')
train_raw.head()

,Master_Index,County,Health level,Years of Experience,Prompt,Nursing Competency,Clinical Panel,Clinician,GPT4.0,LLAMA,GEMINI,DDX SNOMED
0,ID_VBWWP,Uasin Gishu,Sub-county Hospitals and Nursing Homes,18.0,I am a nurse with 18 years of experience in Ge...,Pediatric Emergency Burns,SURGERY,Summary:\nA 4 year old with 5% superficial bur...,Given your vast experience as a nurse in Uasin...,1. Immediate treatment protocol for second-deg...,Here's a response addressing the questions reg...,288514009 | Burn involving 5 percent of body s...
1,ID_XMBBY,Uasin Gishu,National Referral Hospitals,17.0,I am a nurse with 17 years of experience in Ge...,Child Health,PAEDIATRICS,Summary\n6-year-old present with vomiting and ...,Clinical Summary: • A 6-year-old girl w...,Based on the symptoms and signs you've describ...,"Based on the presentation, the 6-year-old girl...",420270002 | Ketoacidosis due to type 1 diabete...
2,ID_JZNZW,Kiambu,Sub-county Hospitals and Nursing Homes,12.0,I am a nurse with 12 years of experience in Ge...,General Emergency,INTERNAL MEDICINE,Summary\nA 47-year-old man presents with sever...,"In this case, you're dealing with a 47-year-ol...","Firstly, I must commend you on your thorough h...","This 47-year-old male presenting with severe, ...",13200003 | Peptic ulcer (disorder)\n25458004 |...
3,ID_QOQTK,Uasin Gishu,National Referral Hospitals,12.0,I am a nurse with 12 years of experience in Pr...,Critical Care,INTERNAL MEDICINE,SUMMARY\n\n72-year-old female with inability t...,"Given ER's clinical presentation and vitals, t...",to me with this query. Based on the informatio...,This 92-year-old female patient (ER) presents ...,14760008 | Constipation (finding)\n419284004 |...
4,ID_ZFJBM,Uasin Gishu,National Referral Hospitals,16.0,I am a nurse with 16 years of experience in Ge...,Adult Health,INTERNAL MEDICINE,"A 22 year old female presents with headache, d...",The 22-year-old female patient is presenting w...,Thank you for presenting this case. Based on t...,This 22-year-old female patient presents with ...,95874006 | Carbon monoxide poisoning from fire...


In [ ]:
train_csv = pd.read_csv('/content/drive/MyDrive/케냐 임상시험 데이터셋/train.csv')
train_csv.head()

,Master_Index,County,Health level,Years of Experience,Prompt,Nursing Competency,Clinical Panel,Clinician,GPT4.0,LLAMA,GEMINI,DDX SNOMED
0,ID_VBWWP,uasin gishu,sub county hospitals and nursing homes,18.0,i am a nurse with 18 years of experience in ge...,pediatric emergency burns,surgery,summary a 4 year old with 5 superficial burns ...,given your vast experience as a nurse in uasin...,1 immediate treatment protocol for second degr...,here s a response addressing the questions reg...,288514009 burn involving 5 percent of body sur...
1,ID_XMBBY,uasin gishu,national referral hospitals,17.0,i am a nurse with 17 years of experience in ge...,child health,paediatrics,summary 6 year old present with vomiting and a...,clinical summary • a 6 year old girl with know...,based on the symptoms and signs you ve describ...,based on the presentation the 6 year old girl ...,420270002 ketoacidosis due to type 1 diabetes ...
2,ID_JZNZW,kiambu,sub county hospitals and nursing homes,12.0,i am a nurse with 12 years of experience in ge...,general emergency,internal medicine,summary a 47 year old man presents with severe...,in this case you re dealing with a 47 year old...,firstly i must commend you on your thorough hi...,this 47 year old male presenting with severe r...,13200003 peptic ulcer disorder 25458004 acute ...
3,ID_QOQTK,uasin gishu,national referral hospitals,12.0,i am a nurse with 12 years of experience in pr...,critical care,internal medicine,summary 72 year old female with inability to w...,given er s clinical presentation and vitals th...,to me with this query based on the information...,this 92 year old female patient er presents wi...,14760008 constipation finding 419284004 altere...
4,ID_ZFJBM,uasin gishu,national referral hospitals,16.0,i am a nurse with 16 years of experience in ge...,adult health,internal medicine,a 22 year old female presents with headache di...,the 22 year old female patient is presenting w...,thank you for presenting this case based on th...,this 22 year old female patient presents with ...,95874006 carbon monoxide poisoning from fire d...


### 피처 설명
0. Master_Index: 인덱스 (str)
1. County: 근무중인 카운티 (str)
2. Health level: 의료기관 유형 (str)
3. Years of Experience: 경력 (float)
4. Prompt: 간호사 소개 + 임상 상황 + 질문 (str)
5. Nursing Compatency: 간호 역량 영역 (str)
6. Clinical Panel: 빈여트를 평가할 의료 전문 분야 (str)
7. Clinician: 인간 임상의의 답변 (str)
8. GPT4.0: GPT-4.0이 생성한 답변 (str)
9. LLAMA: LLAMA가 생성한 답변 (str)
10. GEMINI: GEMINI가 생성한 답변 (str)
11. DDX SNOMED: SNOMED CT 진단 코드 (str)



In [ ]:
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Master_Index         400 non-null    object 
 1   County               400 non-null    object 
 2   Health level         400 non-null    object 
 3   Years of Experience  300 non-null    float64
 4   Prompt               400 non-null    object 
 5   Nursing Competency   400 non-null    object 
 6   Clinical Panel       400 non-null    object 
 7   Clinician            400 non-null    object 
 8   GPT4.0               400 non-null    object 
 9   LLAMA                400 non-null    object 
 10  GEMINI               400 non-null    object 
 11  DDX SNOMED           399 non-null    object 
dtypes: float64(1), object(11)
memory usage: 37.6+ KB


In [ ]:
train_csv['DDX SNOMED'][0]

'288514009 burn involving 5 percent of body surface disorder'

### 결측치 확인

In [ ]:
train_csv.isna().sum()

,0
Master_Index,0
County,0
Health level,0
Years of Experience,100
Prompt,0
Nursing Competency,0
Clinical Panel,0
Clinician,0
GPT4.0,0
LLAMA,0


In [ ]:
nan_row = train_csv[train_csv['Years of Experience'].isna()]
nan_row.reset_index(drop=True, inplace=True)
nan_row

,Master_Index,County,Health level,Years of Experience,Prompt,Nursing Competency,Clinical Panel,Clinician,GPT4.0,LLAMA,GEMINI,DDX SNOMED
0,ID_SUOCB,uasin gishu,national referral hospitals,NaN,i am a nurse working in a national referral ho...,child health,surgery,a 10 year old presenting with lump fever and l...,the presentation of a 10 year old boy with a l...,based on the symptoms and x ray findings i ll ...,given the presentation of a 10 year old boy wi...,307576001 osteosarcoma of bone disorder 601680...
1,ID_LPHGS,kakamega,health centres,NaN,i am a nurse working in a health centres in ka...,general emergency,surgery,summary 19yr old boy involved in rta on examin...,given the patient s history and presentation s...,given the patient s history of a road accident...,you absolutely should not stitch the wound tre...,66551002 psychogenic headache finding 10844110...
2,ID_DBLHH,uasin gishu,health centres,NaN,i am a nurse working in a health centres in ua...,sexual and reproductive health,obstetrics and gynaecology,summary a 26 year old female with complaints o...,based on the symptoms and history provided the...,thank you for presenting this case based on th...,this 26 year old female is presenting with cla...,419760006 bacterial vaginosis disorder 2259560...
3,ID_GMPJA,kakamega,health centres,NaN,i am a nurse working in a health centres in ka...,emergency care gbv,internal medicine,summary a 32 year old reports being in abusive...,yes it is absolutely appropriate for the healt...,this is a very sensitive and critical situatio...,yes it is absolutely proper and ethically requ...,NaN
4,ID_GULKY,kiambu,dispensaries and private clinics,NaN,i am a nurse working in a dispensaries and pri...,maternal and child health,paediatrics,summary 1 month old baby wet umbilical cord dx...,the scenario involves a 1 month old baby with ...,this is a concerning presentation and you mana...,this case presents a concerning situation a on...,239095007 omphalitis disorder 91302008 sepsis ...
...,...,...,...,...,...,...,...,...,...,...,...,...
95,ID_VNCRE,uasin gishu,national referral hospitals,NaN,i am a nurse working in a national referral ho...,child health,paediatrics,summary 1 year old boy with vomiting and fever...,what could be causing recurrent fever and vomi...,given the child s history of recurrent fever a...,this 1 year old boy presents with recurrent fe...,61462000 malaria disorder 91302008 sepsis diso...
96,ID_YMSLH,uasin gishu,national referral hospitals,NaN,i am a nurse working in a national referral ho...,maternal and child health,obstetrics and gynaecology,summary a 28 year old woman admitted at 38 wee...,thank you for providing the detailed case this...,1 is it recommended to do ict test to all the ...,let s address your questions regarding this pr...,199582007 rhesus isoimmunization delivered dis...
97,ID_QMUCF,kakamega,dispensaries and private clinics,NaN,i am a nurse working in a dispensaries and pri...,sexual and reproductive health,obstetrics and gynaecology,summary 26 year old came to the clinic with pr...,case analysis a 26 year old client presents wi...,given the client s symptoms and history there ...,this 26 year old patient presenting with three...,82536003 abnormal hormonal requirement finding...
98,ID_JNZVU,uasin gishu,national referral hospitals,NaN,i am a nurse working in a national referral ho...,sexual and reproductive health,internal medicine,summary a 68 year old male presents with fever...,for a 38 year old para 3 woman presenting with...,based on the symptoms you ve described it s es...,this 38 year old woman presents with concernin...,95883001 bacterial meningitis disorder 5817000...


In [ ]:
nan_row['Prompt'][0]

'i am a nurse working in a national referral hospitals in uasin gishu county in kenya a 10 year old boy presented with a lump fever and limited range of motion in his left hip after a minor trauma x rays of the pelvis and left hip revealed a lytic lesion t 38 bp 110 60 mmhg p 100 b m r 26 b m what laboratory investigation can be done what is the diagnosis what s the management'

In [ ]:
train_csv[train_csv['DDX SNOMED'].isna()]

,Master_Index,County,Health level,Years of Experience,Prompt,Nursing Competency,Clinical Panel,Clinician,GPT4.0,LLAMA,GEMINI,DDX SNOMED
18,ID_GMPJA,kakamega,health centres,NaN,i am a nurse working in a health centres in ka...,emergency care gbv,internal medicine,summary a 32 year old reports being in abusive...,yes it is absolutely appropriate for the healt...,this is a very sensitive and critical situatio...,yes it is absolutely proper and ethically requ...,NaN


## Test data

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/에딥_팀프로젝트/test.csv')
test_data.head()

,Master_Index,County,Health level,Years of Experience,Prompt,Nursing Competency,Clinical Panel
0,ID_CUAOY,uasin gishu,sub county hospitals and nursing homes,2.0,i am a nurse with 2 years of experience in gen...,adult health,surgery ent
1,ID_OGSAY,kiambu,sub county hospitals and nursing homes,22.0,i am a nurse with 22 years of experience in ge...,child health,surgery
2,ID_TYHSA,uasin gishu,national referral hospitals,NaN,i am a nurse working in a national referral ho...,general emergency,internal medicine
3,ID_CZXLD,kakamega,dispensaries and private clinics,NaN,i am a nurse working in a dispensaries and pri...,child health,paediatrics
4,ID_ZJQUQ,kakamega,health centres,NaN,i am a nurse working in a health centres in ka...,child health,paediatrics


1. Years of Experience 결측치(100)개 처리 방법
2. 데이터 증강 필요

# Baseline modeling

In [ ]:
train_csv.head()

,Master_Index,County,Health level,Years of Experience,Prompt,Nursing Competency,Clinical Panel,Clinician,GPT4.0,LLAMA,GEMINI,DDX SNOMED
0,ID_VBWWP,uasin gishu,sub county hospitals and nursing homes,18.0,i am a nurse with 18 years of experience in ge...,pediatric emergency burns,surgery,summary a 4 year old with 5 superficial burns ...,given your vast experience as a nurse in uasin...,1 immediate treatment protocol for second degr...,here s a response addressing the questions reg...,288514009 burn involving 5 percent of body sur...
1,ID_XMBBY,uasin gishu,national referral hospitals,17.0,i am a nurse with 17 years of experience in ge...,child health,paediatrics,summary 6 year old present with vomiting and a...,clinical summary • a 6 year old girl with know...,based on the symptoms and signs you ve describ...,based on the presentation the 6 year old girl ...,420270002 ketoacidosis due to type 1 diabetes ...
2,ID_JZNZW,kiambu,sub county hospitals and nursing homes,12.0,i am a nurse with 12 years of experience in ge...,general emergency,internal medicine,summary a 47 year old man presents with severe...,in this case you re dealing with a 47 year old...,firstly i must commend you on your thorough hi...,this 47 year old male presenting with severe r...,13200003 peptic ulcer disorder 25458004 acute ...
3,ID_QOQTK,uasin gishu,national referral hospitals,12.0,i am a nurse with 12 years of experience in pr...,critical care,internal medicine,summary 72 year old female with inability to w...,given er s clinical presentation and vitals th...,to me with this query based on the information...,this 92 year old female patient er presents wi...,14760008 constipation finding 419284004 altere...
4,ID_ZFJBM,uasin gishu,national referral hospitals,16.0,i am a nurse with 16 years of experience in ge...,adult health,internal medicine,a 22 year old female presents with headache di...,the 22 year old female patient is presenting w...,thank you for presenting this case based on th...,this 22 year old female patient presents with ...,95874006 carbon monoxide poisoning from fire d...


In [ ]:
def preprocess_text(text):
    """Preprocesses the text by lowering case, removing punctuation, and replacing paragraphs with spaces."""
    if not isinstance(text, str):
        return ""
    text = ''.join([c if c.isalnum() or c.isspace() else ' ' for c in text])
    return ' '.join(text.split())

In [ ]:
train_csv['Prompt'] = train_csv['Prompt'].apply(preprocess_text)
train_csv['Clinician'] = train_csv['Clinician'].apply(preprocess_text)
test_data['Prompt'] = test_data['Prompt'].apply(preprocess_text)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

if torch.cuda.device_count() > 1:
    print(f'using {torch.cuda.device_count()} GPU')
    model = torch.nn.DataParallel(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    inputs = tokenizer(
        ["summarize: " + text for text in examples['Prompt']],
        max_length=512,
        padding='max_length',
        truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['Clinician'],
            max_length=128,
            padding='max_length',
            truncation=True
        )

    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': labels['input_ids']
    }

In [ ]:
train_dataset = Dataset.from_pandas(train_csv[['Prompt', 'Clinician']])
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['Prompt', 'Clinician'])

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
train_val_split = train_dataset.train_test_split(test_size=0.01)
train_dataset = train_val_split['train']
eval_dataset = train_val_split['test']

In [ ]:
!pip install evaluate
!pip install -q rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import numpy as np
import evaluate          # ← 🤗  evaluate 라이브러리 (pip install evaluate)

rouge = evaluate.load("rouge")              # 한 번만 로드

def compute_metrics(eval_pred):
    preds, labels = eval_pred

    # 1) logits → token IDs
    if isinstance(preds, tuple):
        preds = preds[0]
    if preds.ndim == 3:                      # [B, L, V]
        preds = np.argmax(preds, axis=-1)

    # 2) 디코딩
    decoded_preds  = tokenizer.batch_decode(preds,   skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 3) ROUGE 계산 (여기선 ROUGE-L F1 하나만 예시)
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    rouge_l_f1 = result["rougeL"]            # 0~1 사이 값

    return {"rougeL_f1": rouge_l_f1}



In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    eval_strategy= "epoch",  # Changed to epoch
    save_strategy= "epoch",  # Changed to epoch
    logging_strategy='steps',
    logging_steps=100,
    save_total_limit=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model='rougeL_f1',  # Set accuracy as the metric for best model
    greater_is_better=True,
    remove_unused_columns=False,
    logging_dir='./logs',
    report_to='none',
    disable_tqdm=False,
    logging_first_step=True,
    prediction_loss_only=False
)


# Create Trainer with progress callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics  # Add metrics computation
)

# Start training with clear progress indication
print("Starting training...")
print(f"Training on {len(train_dataset)} samples, validating on {len(eval_dataset)} samples")
print(f"Batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Total optimization steps: {len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")

trainer.train()


Starting training...
Training on 396 samples, validating on 4 samples
Batch size: 32
Total optimization steps: 36


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rougel F1
1,7.204000,6.440713,0.258410
2,7.204000,6.386612,0.257902


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=36, training_loss=6.304756667878893, metrics={'train_runtime': 61.2564, 'train_samples_per_second': 19.394, 'train_steps_per_second': 0.588, 'total_flos': 150500083236864.0, 'train_loss': 6.304756667878893, 'epoch': 2.808080808080808})

In [ ]:
def generate_predictions(prompts):
    model_to_use = model.module if isinstance(model, torch.nn.DataParallel) else model

    inputs = tokenizer(
        ["summarize: " + text for text in prompts],
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=512
    ).to(next(model_to_use.parameters()).device)  # Ensure inputs are moved to the same device as the model

    with torch.no_grad():
        outputs = model_to_use.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=128
        )

    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]



# Save predictions
test_data['Clinician'] = generate_predictions(test_data['Prompt'].tolist())
test_data[['Master_Index', 'Clinician']].to_csv('submission.csv', index=False)
print("\nTraining complete and predictions saved!")


Training complete and predictions saved!


In [ ]:
test_data.loc[:, ['Prompt', 'Clinician']]

,Prompt,Clinician
0,i am a nurse with 2 years of experience in gen...,a 24 year old female complains of sharp pain i...
1,i am a nurse with 22 years of experience in ge...,a 3 year old boy was brought to the facility h...
2,i am a nurse working in a national referral ho...,a 22 year old man was brought in by the mother...
3,i am a nurse working in a dispensaries and pri...,a 6 year old girl is brought to opd with histo...
4,i am a nurse working in a health centres in ka...,kenyan nurse a nurse working in a health centr...
...,...,...
95,i am a nurse with 29 years of experience in ge...,a child 6 years old came to the clinic accompa...
96,i am a nurse with 6 years of experience in gen...,a 49 year old lady presented at the facility w...
97,i am a nurse with 12 years of experience in ge...,35-year-old man has been trying to conceive wi...
98,i am a nurse with 18 years of experience in ge...,a 48 year old was brought from theatre post th...


# 데이터 증강

In [ ]:
!pip install -q "openai>=1.2" tqdm

In [ ]:
os.environ['OPENAI_API_KEY'] = # API KEY 입력
MODEL_NAME   = "gpt-4o-mini"
MAX_CONC     = 50
BACKOFF_SEC  = 1
BATCH_SAVE   = 10
OUT_FILE     = "augmented_data.csv"

client = AsyncOpenAI()

PROMPT = textwrap.dedent("""
    You are a medical writing assistant.
    - Rephrase the following text so that the meaning stays the same,
      but about 20–30 % of the wording changes.
    - Keep all vital-sign numbers and lab values exactly as written.
    - Do NOT invent new findings or omit existing ones.

    --- Original Text ---
    {text}
    ---------------------
    ### Paraphrased Version:
""").strip()

import os, asyncio, json, pandas as pd, textwrap, time
from tqdm.asyncio import tqdm
from openai import AsyncOpenAI

sem = asyncio.Semaphore(MAX_CONC)

async def call_llm(src: str, temp=0.6, retry=4) -> str | None:
    """LLM 한 번 호출, 실패 시 백오프 재시도."""
    for _ in range(retry):
        try:
            rsp = await client.chat.completions.create(
                model       = MODEL_NAME,
                messages    = [{"role":"user",
                                "content": PROMPT.format(text=src)}],
                temperature = temp,
                max_tokens  = 128,
            )
            return rsp.choices[0].message.content.strip()
        except Exception as e: # 429 포함 모든 예외
            await asyncio.sleep(BACKOFF_SEC)
    return None

# ----------------- 3. 메인 파이프라인 -----------------

async def worker(src: str):
    async with sem:
        tgt = await call_llm(src)
    if tgt:
        pairs.append({"Prompt": src, "augmented_prompt": tgt})
        if len(pairs) % BATCH_SAVE == 0:      # 중간 저장
            pd.DataFrame(pairs).to_csv(OUT_FILE, sep=",", index=False)

async def main(sent_list):
    tasks = [worker(s) for s in sent_list]
    for f in tqdm.as_completed(tasks, total=len(tasks)):
        await f

In [ ]:
original_prompt = train_csv['Prompt']
sentences = original_prompt.tolist()

In [ ]:
import asyncio, time, math, pandas as pd

pairs = []
batch_size = 400
wait = 10

for i in range(1):
    start = time.perf_counter()
    batch = sentences[i*batch_size : (i+1)*batch_size]

    print(f"▶ Batch {i+1}: 문장 {len(batch)}개")
    await main(batch)
    print(f"  누적 쌍: {len(pairs)}")

    # 70초 되기까지 대기(500 rpm 초과시 속도가 엄청 느려짐)
    elapsed = time.perf_counter() - start
    sleep_for = max(0, wait - elapsed)
    if i < math.ceil(len(sentences) / batch_size) - 1:
        print(f"  {sleep_for:.1f}초 대기 후 다음 배치…")
        await asyncio.sleep(sleep_for)

# 최종 저장
pd.DataFrame(pairs).to_csv(OUT_FILE, sep=",", index=False)
print("최종 쌍 수:", len(pairs))

▶ Batch 1: 문장 400개


100%|██████████| 400/400 [00:42<00:00,  9.37it/s]

  누적 쌍: 400
최종 쌍 수: 400


In [ ]:
augmented_pairs = pd.read_csv('augmented_data.csv')
augmented_pairs

,Prompt,augmented_prompt
0,i am a nurse working in a sub county hospitals...,I am a nurse employed in sub-county hospitals ...
1,i am a nurse with 15 years of experience in ge...,"I have 15 years of experience as a nurse, prim..."
2,i am a nurse with 10 years of experience in ge...,I am a nurse with a decade of experience in ge...
3,i am a nurse with 1 years of experience in gen...,I am a nurse with one year of experience in ge...
4,i am a nurse with 15 years of experience in me...,I am a nurse with 15 years of expertise in men...
...,...,...
395,i am a nurse working in a dispensaries and pri...,I am a nurse employed in dispensaries and priv...
396,i am a nurse with 19 years of experience in ge...,I am a nurse with 19 years of experience in ge...
397,i am a nurse working in a dispensaries and pri...,I am a nurse employed at dispensaries and priv...
398,i am a nurse with 3 years of experience in gen...,I am a nurse with three years of experience in...


In [ ]:
train_csv.columns

Index(['Master_Index', 'County', 'Health level', 'Years of Experience',
       'Prompt', 'Nursing Competency', 'Clinical Panel', 'Clinician', 'GPT4.0',
       'LLAMA', 'GEMINI', 'DDX SNOMED'],
      dtype='object')

In [ ]:
augmented_dataset = augmented_pairs.merge(train_csv[['Master_Index', 'Prompt', 'Clinician']],
                                          on = 'Prompt',
                                          how = 'left')
aug_df = augmented_dataset.loc[:, ['Master_Index', 'augmented_prompt', 'Clinician']]
aug_df = aug_df.rename(columns={'augmented_prompt': 'Prompt'})
aug_df

,Master_Index,Prompt,Clinician
0,ID_PNWEY,I am a nurse employed in sub-county hospitals ...,summary a 45year old man has a history of conf...
1,ID_OFPVA,"I have 15 years of experience as a nurse, prim...",summary 1 year old baby presenting with ingest...
2,ID_KTWAF,I am a nurse with a decade of experience in ge...,summary a neonate brought in apgar 8 10 10 neo...
3,ID_ZOGTE,I am a nurse with one year of experience in ge...,summary a 15 year old boy was brought to the f...
4,ID_XROUJ,I am a nurse with 15 years of expertise in men...,summary a 66 year old male with diabetes melli...
...,...,...,...
395,ID_ZBFGY,I am a nurse employed in dispensaries and priv...,summary a case of expectant mother with one pr...
396,ID_VZVPQ,I am a nurse with 19 years of experience in ge...,summary a 14 year old girl presents with loss ...
397,ID_QMUCF,I am a nurse employed at dispensaries and priv...,summary 26 year old came to the clinic with pr...
398,ID_VGPNK,I am a nurse with three years of experience in...,a 2 year old girl was brought to the hospital ...


In [ ]:
aug_df.head()

,Master_Index,Prompt,Clinician
0,ID_PNWEY,I am a nurse employed in sub-county hospitals ...,summary a 45year old man has a history of conf...
1,ID_OFPVA,"I have 15 years of experience as a nurse, prim...",summary 1 year old baby presenting with ingest...
2,ID_KTWAF,I am a nurse with a decade of experience in ge...,summary a neonate brought in apgar 8 10 10 neo...
3,ID_ZOGTE,I am a nurse with one year of experience in ge...,summary a 15 year old boy was brought to the f...
4,ID_XROUJ,I am a nurse with 15 years of expertise in men...,summary a 66 year old male with diabetes melli...


In [ ]:
aug_dataset = Dataset.from_pandas(aug_df[['Prompt', 'Clinician']])
aug_dataset = aug_dataset.map(tokenize_function, batched=True, remove_columns=['Prompt', 'Clinician'])

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from datasets import Dataset, concatenate_datasets
# validation set을 제외한 train set에 concat
train_dataset = concatenate_datasets([train_dataset, aug_dataset])
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 796
})

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq

# ── Seq2Seq 전용 collator 정의 ─────────────────────────────────
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=tokenizer.pad_token_id,
    pad_to_multiple_of=8       # fp16 시 성능 향상 옵션 (선택)
)

# → TrainingArguments → Seq2SeqTrainingArguments 로 바꿔 주고
training_args = Seq2SeqTrainingArguments(
    output_dir          = './results',
    per_device_train_batch_size = 4,
    per_device_eval_batch_size  = 4,
    gradient_accumulation_steps = 8,
    num_train_epochs    = 3,
    eval_strategy = "epoch",
    save_strategy       = "epoch",
    logging_strategy    = 'steps',
    logging_steps       = 100,
    save_total_limit    = 2,
    learning_rate       = 2e-5,
    weight_decay        = 0.01,
    warmup_steps        = 500,
    fp16                = True,
    load_best_model_at_end  = True,
    metric_for_best_model   = 'rougeL_f1',
    greater_is_better       = True,
    remove_unused_columns   = False,
    logging_dir             = './logs',
    report_to               = 'none',

    # ── 이 두 줄이 Seq2Seq 전용 옵션
    predict_with_generate   = True,
    generation_max_length    = 128,
)

trainer = Seq2SeqTrainer(
    model            = model,
    args             = training_args,
    train_dataset    = train_dataset,
    eval_dataset     = eval_dataset,
    data_collator    = data_collator,    # Seq2Seq용 collator
    tokenizer        = tokenizer,        # generate()에 필요
    compute_metrics  = compute_metrics,
)

# Start training with clear progress indication
print("Starting training...")
print(f"Training on {len(train_dataset)} samples, validating on {len(eval_dataset)} samples")
print(f"Batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Total optimization steps: {len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")

trainer.train()

<ipython-input-128-ec9a9dc3ac3d>:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Starting training...
Training on 796 samples, validating on 4 samples
Batch size: 32
Total optimization steps: 72


Epoch,Training Loss,Validation Loss,Rougel F1
0,No log,4.220767,0.261699
1,No log,4.192640,0.261699
2,No log,4.148880,0.261699


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=72, training_loss=4.682816823323567, metrics={'train_runtime': 80.1143, 'train_samples_per_second': 29.807, 'train_steps_per_second': 0.899, 'total_flos': 316158448238592.0, 'train_loss': 4.682816823323567, 'epoch': 2.964824120603015})

In [ ]:
def generate_predictions(prompts):
    model_to_use = model.module if isinstance(model, torch.nn.DataParallel) else model

    inputs = tokenizer(
        ["summarize: " + text for text in prompts],
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=512
    ).to(next(model_to_use.parameters()).device)  # Ensure inputs are moved to the same device as the model

    with torch.no_grad():
        outputs = model_to_use.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=128
        )

    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]



# Save predictions
test_data['Clinician'] = generate_predictions(test_data['Prompt'].tolist())
test_data[['Master_Index', 'Clinician']].to_csv('submission.csv', index=False)
print("\nTraining complete and predictions saved!")


Training complete and predictions saved!


In [ ]:
test_data.loc[:, ['Prompt', 'Clinician']]

,Prompt,Clinician
0,i am a nurse with 2 years of experience in gen...,"general nursing nurse in uasin gishu county, k..."
1,i am a nurse with 22 years of experience in ge...,general nursing is a sub county hospital and n...
2,i am a nurse working in a national referral ho...,a 22 year old man was brought in by the mother...
3,i am a nurse working in a dispensaries and pri...,a 6 year old girl is brought to opd with histo...
4,i am a nurse working in a health centres in ka...,kenyan nurse a nurse working in a health centr...
...,...,...
95,i am a nurse with 29 years of experience in ge...,general nursing nurse with 29 years of experie...
96,i am a nurse with 6 years of experience in gen...,"general nursing in kakamega county, kenya, has..."
97,i am a nurse with 12 years of experience in ge...,general nursing worker has 12 years of experie...
98,i am a nurse with 18 years of experience in ge...,


In [ ]:
import pandas as pd
pd.read_csv('/content/augmented_data.csv')

,Prompt,augmented_prompt
0,i am a nurse working in a sub county hospitals...,I am a nurse employed in sub-county hospitals ...
1,i am a nurse with 15 years of experience in ge...,"I have 15 years of experience as a nurse, prim..."
2,i am a nurse with 10 years of experience in ge...,I am a nurse with a decade of experience in ge...
3,i am a nurse with 1 years of experience in gen...,I am a nurse with one year of experience in ge...
4,i am a nurse with 15 years of experience in me...,I am a nurse with 15 years of expertise in men...
...,...,...
395,i am a nurse working in a dispensaries and pri...,I am a nurse employed in dispensaries and priv...
396,i am a nurse with 19 years of experience in ge...,I am a nurse with 19 years of experience in ge...
397,i am a nurse working in a dispensaries and pri...,I am a nurse employed at dispensaries and priv...
398,i am a nurse with 3 years of experience in gen...,I am a nurse with three years of experience in...
